In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import skimage.io
import skimage.morphology

import os.path

import sys
sys.path.insert(0, '/home/jr0th/github/segmentation/code/')
import helper.metrics

import numpy as np

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

Using TensorFlow backend.


In [2]:
cp_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand_200/CP_labels/'
cp_label_tiff = '/home/jr0th/github/segmentation/experiments/BBBC022_hand_200/CP_labels_tiff/'
gt_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand_200/GT_labels/'

mat_cp_dir = '/home/jr0th/github/segmentation/experiments/BBBC022_hand_200/IoU_CP/'

In [3]:
# make CP labels png and relabel due to weird numbering
labels_cp_tiff = skimage.io.imread_collection(cp_label_tiff + "*.tiff")

# loop over all images
for index in range(len(labels_cp_tiff)):
    
    path = labels_cp_tiff.files[index]
    filename = os.path.basename(path)
    filename_wo_ext = os.path.splitext(filename)[0]
    
    label_cp_tiff = labels_cp_tiff[index]
    label_cp_png = skimage.morphology.label(label_cp_tiff)
    label_cp_png = skimage.img_as_ubyte(label_cp_png)
    
    skimage.io.imsave(cp_label + filename_wo_ext + '.png', label_cp_png)

/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:118: UserWarning: Possible sign loss when converting negative image of type int64 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int64 to uint8 without scaling because max value 91 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int64 to uint8 without scaling because max value 63 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int64 to uint8 without scaling because max value 141 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int64 to uint8 without scaling 

/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int64 to uint8 without scaling because max value 112 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))


In [4]:
# buffer for results
results = np.empty(shape = (0, 4))

labels_cp = skimage.io.imread_collection(cp_label + "*.png")

# loop over all images
for index in range(len(labels_cp)):
    
    # print progress
    print(index + 1, '/', len(labels_cp))
    
    path = labels_cp.files[index]
    filename = os.path.basename(path)
    filename_wo_ext = os.path.splitext(filename)[0]
    
    # get labels
    label_cp = labels_cp[index]
    label_gt = skimage.io.imread(gt_label + filename_wo_ext + ".png")
    
    nb_gt_cells = np.max(label_gt)
    
    [nb_overdetection, nb_underdetection, mean_IoU, IoUs] = helper.metrics.compare_two_labels(label_cp, label_gt, True)
    
    np.save(mat_cp_dir + filename_wo_ext, IoUs)
    
    result = np.hstack((nb_overdetection, nb_underdetection, mean_IoU, nb_gt_cells))
    results = np.vstack((results, result))

print(results)

# calculate error ratio
sums = np.sum(results, axis = 0)
nb_gt_cells_total = sums[3]
nb_errors = sums[0] + sums[1]

err_ratio = nb_errors / nb_gt_cells_total
print(err_ratio)

np.savetxt('./results_table_CP.txt', results)
np.savetxt('./results_summary_CP.txt', np.array([nb_errors, nb_gt_cells_total, err_ratio]))

1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50
50 / 50
[[   5.           20.            0.72705078  106.        ]
 [   3.            9.            0.76768798   69.        ]
 [   3.           12.            0.82493889  150.        ]
 [   1.            6.            0.61922258   22.        ]
 [   8.           25.            0.7334581   127.        ]
 [   0.            7.            0.83474422   89.        ]
 [   6.           31.            0.66167587  123.        ]
 [   1.           10.            0.79550165  102.        ]
 [   2.           18.            0.77054763  127.        ]
 [   3.           14.            0.72684669   74.        ]
 [   2.           1